In [1]:
# Встановлення необхідних бібліотек
!pip install transformers datasets accelerate evaluate sentencepiece -q

In [2]:
import torch
import time
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
    pipeline
)
import evaluate
import collections
import matplotlib.pyplot as plt

In [3]:
# Фіксація seed для відтворюваності
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [4]:
# 1. Завантаження датасету
raw_datasets = load_dataset("squad_v2")

# Для швидкості беремо частину даних
train_subset = raw_datasets["train"].shuffle(seed=42).select(range(2000))
val_subset = raw_datasets["validation"].select(range(500))

print("Приклад даних:", train_subset[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Приклад даних: {'id': '56e0f3907aa994140058e80a', 'title': 'Canon_law', 'context': 'The Roman Catholic Church canon law also includes the main five rites (groups) of churches which are in full union with the Roman Catholic Church and the Supreme Pontiff:', 'question': 'What term characterizes the intersection of the rites with the Roman Catholic Church?', 'answers': {'text': ['full union'], 'answer_start': [104]}}


In [5]:
# Функція препроцесингу
def preprocess_function(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second", # Обрізаємо контекст, якщо він задовгий, а не питання
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        # Якщо відповіді немає (SQuAD v2 unanswerable)
        if len(answer["answer_start"]) == 0:
            start_positions.append(0) # CLS token index
            end_positions.append(0)
        else:
            # Знаходимо токени старту і кінця
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Знаходимо межі контексту
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # Якщо відповідь виходить за межі обрізаного контексту -> CLS
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Рухаємось до початку відповіді
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                # Рухаємось до кінця відповіді
                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
results_table = []

def run_experiment(model_checkpoint):
    print(f"\nProcessing: {model_checkpoint}")

    # 1. Токенізація
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    # Виправляємо проблему з DeBERTa tokenizer (іноді потрібен add_prefix_space)
    if "deberta" in model_checkpoint:
         tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

    # Видаляємо старі колонки, щоб уникнути конфліктів
    tokenized_train = train_subset.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=raw_datasets["train"].column_names
    )
    tokenized_val = val_subset.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=raw_datasets["validation"].column_names
    )

    # 2. Модель
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

    # Підрахунок параметрів
    model_params = model.num_parameters() / 1e6
    print(f"Parameters: {model_params:.2f}M")

    # 3. Налаштування тренування
    args = TrainingArguments(
        output_dir=f"{model_checkpoint}-finetuned-squad",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        save_strategy="no",
        fp16=True,
        push_to_hub=False,
    )

    data_collator = DefaultDataCollator()

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # 4. Навчання + Замір часу
    start_time = time.time()
    train_output = trainer.train()
    end_time = time.time()

    training_time = end_time - start_time

    # 5. Оцінка
    eval_metrics = trainer.evaluate()
    val_loss = eval_metrics["eval_loss"]

    # Розмір моделі на диску (приблизний)
    model_size_mb = model_params * 4

    # Збереження результатів
    results_table.append({
        "Модель": model_checkpoint,
        "Параметри (M)": round(model_params, 2),
        "Час навчання (сек)": round(training_time, 2),
        "Validation Loss": round(val_loss, 4),
        "Розмір ваг (MB)": round(model_size_mb, 2)
    })

    return trainer, tokenizer

In [7]:
# API: 31e61c678bbd9ec126928439dcfdf0c39398986e

In [8]:
# Запуск експериментів

# Модель 1: RoBERTa Base
trainer_roberta, tokenizer_roberta = run_experiment("roberta-base")

# Модель 2: DeBERTa v3 Base (сучасніша, зазвичай краща якість)
trainer_deberta, tokenizer_deberta = run_experiment("microsoft/deberta-v3-base")


Processing: roberta-base


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters: 124.06M


/tmp/ipython-input-2859249014.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Currently logged in as: havryliuk-bohdana (havryliuk-bohdana-igor-sikorsky-kyiv-polytechnic-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.762801
2,No log,1.681704



Processing: microsoft/deberta-v3-base


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2859249014.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Parameters: 183.83M


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss
1,No log,2.349463
2,No log,2.160072


In [9]:
# Вивід таблиці
df_results = pd.DataFrame(results_table)
print("\nПорівняння моделей")
display(df_results)


Порівняння моделей


,Модель,Параметри (M),Час навчання (сек),Validation Loss,Розмір ваг (MB)
0,roberta-base,124.06,99.72,1.6817,496.23
1,microsoft/deberta-v3-base,183.83,167.59,2.1601,735.33


In [10]:
# Інференс на власному прикладі
# Використовуємо pipeline для зручності, передаючи навчену модель

def test_inference(model, tokenizer, model_name):
    print(f"\nTesting: {model_name}")
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

    context = """
    Transformers changed the NLP landscape significantly.
    Before transformers, RNNs and LSTMs were the standard.
    Google introduced BERT in 2018.
    However, transformers can be computationally expensive.
    """

    # Питання, на яке Є відповідь
    q1 = "Who introduced BERT?"
    res1 = qa_pipeline(question=q1, context=context)
    print(f"Q: {q1}")
    print(f"A: {res1['answer']} (score: {res1['score']:.4f})")

    # Питання, на яке НЕМАЄ відповіді (SQuAD v2 feature)
    # Pipeline за замовчуванням намагається знайти відповідь,
    # але низький score може свідчити про відсутність відповіді.
    q2 = "When was ChatGPT released?"
    res2 = qa_pipeline(question=q2, context=context)
    print(f"Q: {q2}")
    print(f"A: {res2['answer']} (score: {res2['score']:.4f})")

In [11]:
print("\nІнференс на власних прикладах")
test_inference(trainer_roberta.model, tokenizer_roberta, "RoBERTa")
test_inference(trainer_deberta.model, tokenizer_deberta, "DeBERTa")

Device set to use cuda:0
Device set to use cuda:0



Інференс на власних прикладах

Testing: RoBERTa
Q: Who introduced BERT?
A: Google introduced BERT in 2018 (score: 0.1267)
Q: When was ChatGPT released?
A: 
    (score: 0.0005)

Testing: DeBERTa
Q: Who introduced BERT?
A:  Google (score: 0.4554)
Q: When was ChatGPT released?
A:  BERT in 2018. (score: 0.0372)
